In [ ]:
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN
from PIL import Image 
import numpy as np
import os

from matplotlib.patches import Rectangle
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from scipy.spatial.distance import cosine

In [ ]:
IMAGE_PATH = 'data/samples/my-face.png'
im = Image.open(IMAGE_PATH)
min_size = im.size[0] if im.size[0]<=im.size[1] else im.size[1]
left = int(im.size[0]/2-min_size/2)
upper = int(im.size[1]/2-min_size/2)
right = left + min_size
lower = upper + min_size

im_cropped = im.crop((left, upper,right,lower))
print(im_cropped.size)
im_cropped.show()

In [ ]:
IMAGE_PATH = 'data/samples/boyband.png'
im = Image.open(IMAGE_PATH)

In [ ]:
np_img =  np.array(im_cropped)[:,:,0:3]
np_img.shape

In [ ]:
detector = MTCNN()

faces = detector.detect_faces(np_img)
print(faces)

In [ ]:
def highlight_faces(image, faces,image_path=None):
  # display image
  if image_path:
    image = plt.imread(image_path)
  plt.imshow(image)

  ax = plt.gca()

  # for each face, draw a rectangle based on coordinates
  for face in faces:
    x, y, width, height = face['box']
    face_border = Rectangle((x, y), width, height,
                          fill=False, color='red')
    ax.add_patch(face_border)
  plt.show()

In [ ]:
highlight_faces(np_img, faces)

# Pack It in A Function

In [ ]:
def extract_face_from_image(image_path, required_size=(224, 224),square=False):
  # load image and detect faces
  im = Image.open(image_path)

  if square:
    min_size = im.size[0] if im.size[0]<=im.size[1] else im.size[1]
    left = int(im.size[0]/2-min_size/2)
    upper = int(im.size[1]/2-min_size/2)
    right = left + min_size
    lower = upper + min_size

    im = im.crop((left, upper,right,lower))
  im_cropped =  np.array(im)[:,:,0:3]

  detector = MTCNN()
  faces = detector.detect_faces(im_cropped)

  face_images = []

  for face in faces:
    # extract the bounding box from the requested face
    x1, y1, width, height = face['box']
    x2, y2 = x1 + width, y1 + height

    # extract the face
    face_boundary = im_cropped[y1:y2, x1:x2]

    # resize pixels to the model size
    face_image = Image.fromarray(face_boundary)
    face_image = face_image.resize(required_size)
    face_array = np.asarray(face_image)
    face_images.append(face_array)

  return face_images

In [ ]:
BASE_PATH = 'data/samples'
extracted_face = extract_face_from_image(os.path.join(BASE_PATH,'boyband.png'))

In [ ]:
plt.imshow(extracted_face[8])

In [ ]:
def get_model_scores(faces):
  samples = np.asarray(faces, 'float32')

  # prepare the data for the model
  samples = preprocess_input(samples, version=2)

  # create a vggface model object
  model = VGGFace(model='resnet50',
      include_top=False,
      input_shape=(224, 224, 3),
      pooling='avg')

  # perform prediction
  return model.predict(samples)

In [ ]:
faces = [extract_face_from_image(os.path.join(BASE_PATH,image_path))
         for image_path in ['my-face.png','boyband.png']]

model_scores = [get_model_scores(i) for i in faces]

In [ ]:
model_scores

In [ ]:
# len(model_scores[0][0])
cosine(model_scores[0][0],model_scores[1][0])

In [ ]:
for i in range(len(model_scores[1])):
    cosine_score = cosine(model_scores[0][0], model_scores[1][i])
    if cosine_score <= 0.4:
        print("Face Matched with face {} with a score of {}".format(i,cosine_score))
        fig = plt.figure(figsize=(10, 7))
        fig.add_subplot(1, 2, 1) 
        plt.imshow(faces[0][0])
        fig.add_subplot(1, 2, 2)
        plt.imshow(faces[1][i])
        plt.show()